### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [1]:
import csv
import pandas as pd
import numpy as np
import json

1. label1 回应情况

In [35]:
import os
import pandas as pd

# 读取标签文件

# 1. 回应情况- ok
label1_1 = pd.read_csv('labelA/回应情况-表格 1.csv',header=1).iloc[:,:5]
label1_2 = pd.read_csv('label_2/回应情况-表格 1.csv',header=1 ).iloc[:,:5]
# label1_3 = pd.read_csv('label_3/回应情况-表格 1.csv')

label1 = pd.concat([label1_1, label1_2], axis=0)
label1 = label1.rename(columns={"Unnamed: 0":"切片ID"})




In [36]:
label1.head()

,切片ID,强,中,弱,没有回应（忽视）
0,15YS_20230317_01/VCAM_0000,0,0,1,0
1,15YS_20230317_01/VCAM_0000_1,0,0,1,0
2,15YS_20230317_01/VCAM_0000_2,1,0,0,0
3,15YS_20230317_01/VCAM_0000_3,0,0,1,0
4,15YS_20230317_01/VCAM_0000_4,1,0,0,0


In [17]:
label1.shape

(3997, 6)

In [4]:
label1.iloc[1,0][-4:]

'0000'

2. label2 互动情况积极与消极

In [47]:
# 2.互动情况的消极与积极
label2_1 = pd.read_csv('labelA/互动情况的消极与积极-表格 1.csv',header=1).iloc[:,:6]
label2_2 = pd.read_csv('label_2/互动情况的消极与积极-表格 1.csv',header=1).iloc[:,:6]

label2 = pd.concat([label2_1, label2_2], axis=0)
label2 = label2.rename(columns={"Unnamed: 0":"切片ID"})

# # 合并整体和部分(积极消极情况)
label2['积极'] = label2[['整体积极','部分积极']].sum(axis=1)
label2['消极'] = label2[['整体消极','部分消极']].sum(axis=1)

label2 = label2.drop(['整体积极','部分积极','部分消极','整体消极'],axis=1)

In [48]:
label2.head()

,切片ID,中性,积极,消极
0,15YS_20230317_01/VCAM_0000,0,1,0
1,15YS_20230317_01/VCAM_0000_1,0,1,0
2,15YS_20230317_01/VCAM_0000_2,0,1,0
3,15YS_20230317_01/VCAM_0000_3,0,1,0
4,15YS_20230317_01/VCAM_0000_4,0,1,0


3. label3 任务专注

In [37]:
# 3.任务专注
label3_1 = pd.read_csv('labelA/任务专注-表格 1.csv',header=1).iloc[:,:3]
label3_2 = pd.read_csv('label_2/任务专注-表格 1.csv',header=1).iloc[:,:3]

label3 = pd.concat([label3_1, label3_2], axis=0)
label3 = label3.rename(columns={"Unnamed: 0":"切片ID"})

In [38]:
label3.head()

,切片ID,专注（任务中）,走神（任务外）
0,15YS_20230317_01/VCAM_0000,1,0
1,15YS_20230317_01/VCAM_0000_1,1,0
2,15YS_20230317_01/VCAM_0000_2,1,0
3,15YS_20230317_01/VCAM_0000_3,1,0
4,15YS_20230317_01/VCAM_0000_4,1,0


4. label4 任务内外的互动行为

In [41]:
# 4.互动行为
label4_1_1 = pd.read_csv('labelA/任务外的互动行为-表格 1.csv',header=1).iloc[:,:5]
label4_1_2 = pd.read_csv('labelA/任务中的互动行为-表格 1.csv',header=1).iloc[:,1:5]
label4_2_1 = pd.read_csv('label_2/任务外的互动行为-表格 1.csv',header=1).iloc[:,:5]
label4_2_2 = pd.read_csv('label_2/任务中的互动行为-表格 1.csv',header=1).iloc[:,1:5]

# 行连接
label4_1 = pd.concat([label4_1_1, label4_1_2], axis=1)
label4_2 = pd.concat([label4_2_1, label4_2_2], axis=1)

label4 = pd.concat([label4_1, label4_2], axis=0)
label4 = label4.rename(columns={"Unnamed: 0":"切片ID"})

In [42]:
label4.head()

,切片ID,玩乐,闲聊,一人独立尝试一人摸鱼\n,各自神游,主导,支持,旁观,冲突
0,15YS_20230317_01/VCAM_0000,0,0,0,0,0,0,1,0
1,15YS_20230317_01/VCAM_0000_1,0,0,0,0,0,0,1,0
2,15YS_20230317_01/VCAM_0000_2,0,0,0,0,1,0,0,0
3,15YS_20230317_01/VCAM_0000_3,0,0,0,0,1,0,0,0
4,15YS_20230317_01/VCAM_0000_4,0,0,0,0,1,0,0,0


In [4]:
# missing features

missing =["19YS_20230318_03/VCAM_0017_1", "19YS_20230318_03/VCAM_0016", "19YS_20230318_03/VCAM_0016_1"]

for i in missing:
    label1 = label1[label1['切片ID'] != i]
    label2 = label2[label2['切片ID'] != i]
    label3 = label3[label3['切片ID'] != i]
    label4 = label4[label4['切片ID'] != i]


### 合并读取face和pose特征  
按照视频读取frame

In [5]:
def get_vel_and_acc(features):
    """
    cal the mean and var of velocity and acceleration respectively of input features
    """
    try:
        first_order_derivatives = np.gradient(features, axis=0)
        second_order_derivatives = np.gradient(first_order_derivatives, axis=0)
    except:
        print("Error in get_vel_and_acc")
        print(features.shape)
        print(print(f"len:{len(face_df0)} and {len(face_df1)}"))
        print(max_fm)
        print("------------------")

    # velecity
    vel_means = np.mean(first_order_derivatives, axis=0)
    vel_var = np.var(first_order_derivatives, axis=0)

    # acceleration
    acc_means = np.mean(second_order_derivatives, axis=0)
    acc_var = np.var(second_order_derivatives, axis=0)
    
    return vel_means, vel_var, acc_means, acc_var


In [6]:
# set openface confidence threshold
confidence_threshold = 0.1

# 导入特征并对齐标签
low_video_features = []
high_video_features = []
video_labels = []
labels = []
frame2video_id = [] # 记录frame属于哪个video
# 记录原本有多少条记录
face_count = 0
pose_count = 0
# 记录读入的记录
rface_count = 0
rpose_count = 0

# for each video
for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    frame_features = []
    try:
        # VCAM number
        Date = directory.split('/')[0] # 15YS_20230317_01
        Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
        VcamID = Vcam.split('_')[1] # xxxx

        # reconstruct the correct Openpose directory
        video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
        json_directory = os.path.join(video_directory, 'json')
    except:
        print(f"[load error]:{directory}")

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    

    # the correct Openface csv file
    # feature_file = os.path.join('siblings_face', directory + '.csv')
    feature_file = os.path.join('siblings_face_1024', Vcam + '.csv')


    # the label data for this video
    label_value = label.iloc[i, 1:] # 取1-4位的标签
    video_id = i # video id

    # Load face data of a video
    try:
        face_csv_data = pd.read_csv(feature_file)
    except:
        print(f"[face error]: {feature_file} not exist")
        continue
    
    # log the num of face data rows
    face_count += face_csv_data.shape[0]

    # 丢弃置信度小于0.5的数据
    face_csv_data = face_csv_data[face_csv_data['confidence'] >= confidence_threshold]

    # TODO 丢弃 success = 0 的数据
    face_csv_data = face_csv_data[face_csv_data['success'] ==1]

    # 找到frame中有多于2个不同face_id的帧
    counts = face_csv_data.groupby('frame')['face_id'].nunique()
    frames_to_drop = counts[counts > 2].index.tolist()

    
    # 对于每个需要处理的帧，删除置信度最低的行，直到只剩下2个face_id
    for frame in frames_to_drop:
        frame_df = face_csv_data[face_csv_data['frame'] == frame]
        while frame_df['face_id'].nunique() > 2:
            min_confidence = frame_df['confidence'].min()
            rows_to_drop = frame_df[(frame_df['confidence'] == min_confidence)]['face_id'].tolist()
            frame_df = frame_df[~frame_df['face_id'].isin(rows_to_drop)]
        face_csv_data = face_csv_data[face_csv_data['frame'] != frame]
        face_csv_data = pd.concat([face_csv_data, frame_df], ignore_index=True)

    # 丢弃只有1个face_id的帧 
    # TODO 是否可以不丢弃？
    face_csv_data = face_csv_data.groupby('frame').filter(lambda x: len(x) == 2)


    # 按照frame分组，并根据x_30(nose x位置)的大小对face_id进行赋值 x较小标记为0
    def assign_face_id(group):
        if len(group) != 2:
            return None
        else:
            if group['x_30'].iloc[0] < group['x_30'].iloc[1]:
                group['face_id'] = [0, 1]
            else:
                group['face_id'] = [1, 0]
            return group

    face_csv_data = face_csv_data.groupby('frame').apply(assign_face_id).reset_index(drop=True)
    # # 重新排序
    # face_csv_data= face_csv_data.set_index(['face_id', 'frame'])

    # def sort_by_frame(group):
    #     return group.sort_values(by='frame')

    # face_csv_data = face_csv_data.groupby('face_id').apply(sort_by_frame)


    # 将df根据face_id列分成两个DataFrame对象
    face_df0 = face_csv_data[face_csv_data['face_id'] == 0]
    face_df1 = face_csv_data[face_csv_data['face_id'] == 1]

    # 按照frame_id列进行排序
    # face_df0 = face_df0.sort_values(by='frame')
    # face_df1 = face_df1.sort_values(by='frame')




    # 将df的索引设置为frame_id,并且插值
    try:
        min_fm, max_fm = face_df0['frame'].min(), face_df0['frame'].max()
        if(max_fm - min_fm >2):
            # 记录contribute 数据数量
            rface_count += 2*face_df0.shape[0]
            # 插值
            face_df0 = face_df0.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
            face_df1 = face_df1.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
        else:
            print(f"[face error] in {feature_file}: not enough frames")
            # print(f"len:{len(face_df0)} and {len(face_df1)}")
            continue
    except:
        print(f"[face error] in {feature_file}:unknown error in interpolation and reindex")
        continue
    # # 对df进行插值
    #  face_df0.reset_index()
    
    ###############  face low feature #######################
    # low level feat to cal vel and acc
    face_low_feat = ['pose_Tx', 'pose_Ty', 'pose_Tz','pose_Rx', 'pose_Ry', 'pose_Rz', 'gaze_angle_x', 'gaze_angle_y']
    au_feat = ['AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU15_r',  'AU20_r', 'AU23_r', 'AU26_r']
    

    # 每个人的low feat 的 vel and acc
    v_mean_0, v_var_0, a_mean_0, a_var_0 = get_vel_and_acc(face_df0[face_low_feat])
    v_mean_1, v_var_1, a_mean_1, a_var_1 = get_vel_and_acc(face_df1[face_low_feat])
    au_mean_0 = np.array(face_df0[au_feat].mean())
    au_mean_1 = np.array(face_df1[au_feat].mean())  
    au_var_0 = np.array(face_df0[au_feat].var())
    au_var_1 = np.array(face_df1[au_feat].var())


    # 每一frame的 face feature
    # features_face = face_csv_data.values 
    low_features_face_0 = np.concatenate((v_mean_0, v_var_0, a_mean_0, a_var_0,au_mean_0,au_var_0))
    low_features_face_1 = np.concatenate((v_mean_1, v_var_1, a_mean_1, a_var_1,au_mean_1,au_var_1))

    ###############  face high feature #######################

    # @1 gaze_angle -> indicating gaze direction
    gaze_angle_0 = np.array(face_df0[['gaze_angle_x','gaze_angle_y']])
    gaze_angle_1 = np.array(face_df1[['gaze_angle_x','gaze_angle_y']])

    gaze_mean_0 = gaze_angle_0.mean(axis=0)
    gaze_mean_1 = gaze_angle_1.mean(axis=0) 
    gaze_var_0 = gaze_angle_0.var(axis=0)
    gaze_var_1 = gaze_angle_0.var(axis=0)


    # @2 计算两个人之间的角度夹角
    # gaze_diff_x = face_df0['gaze_angle_x'] - face_df1['gaze_angle_x']
    # gaze_diff_y = face_df0['gaze_angle_y'] - face_df1['gaze_angle_y']

    cos_angles = np.sum(gaze_angle_0 * gaze_angle_1, axis=1) / (np.linalg.norm(gaze_angle_0, axis=1) * np.linalg.norm(gaze_angle_1, axis=1))
    angles = np.arccos(cos_angles)
    # 将弧度值转换为角度值
    gaze_angles = np.degrees(angles)

    gaze_angle_mean = gaze_angles.mean(axis=0)
    gaze_angle_var = gaze_angles.var(axis=0) 
    v_mean_ga, v_var_ga, a_mean_ga, a_var_ga = get_vel_and_acc(gaze_angles)
    # 单个数值合并成 array
    gaze_feat = np.array([gaze_angle_mean, gaze_angle_var, v_mean_ga, v_var_ga, a_mean_ga, a_var_ga])


    # @3 Motion synchronization -> abs
    v_mean_ms, v_var_ms, a_mean_ms, a_var_ms = get_vel_and_acc((face_df0[face_low_feat] - face_df1[face_low_feat]).abs())


    # 每一frame的 face feature
    # features_face = face_csv_data.values 
    high_features_face = np.concatenate((gaze_mean_0, gaze_mean_1, gaze_var_0, gaze_var_1, \
                                         gaze_feat,
                                         v_mean_ms, v_var_ms, a_mean_ms, a_var_ms
                                        ))


    ###############  pose feature #######################
    
    
    pose_json_data = [] # save all frame data

    # feature names in 'pose_keypoints_2d' in json file
    col_name = [
        'frame','face_id',
       'x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 'x3', 'y3', 'c3', 'x4', 'y4', 'c4',
       'x5', 'y5', 'c5', 'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
       'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11', 'x12', 'y12', 'c12',
       'x13', 'y13', 'c13', 'x14', 'y14', 'c14', 'x15', 'y15', 'c15', 'x16', 'y16', 'c16', 
       'x17', 'y17', 'c17', 'x18', 'y18', 'c18', 'x19', 'y19', 'c19', 'x20', 'y20', 'c20', 
       'x21', 'y21', 'c21', 'x22', 'y22', 'c22', 'x23', 'y23', 'c23', 'x24', 'y24', 'c24'
       ]

    # 创建空的 dataframe
    pose_df0 = pd.DataFrame(columns=col_name)
    pose_df1 = pd.DataFrame(columns=col_name)
    
    
    # Load pose data of a video frame
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
        except:
            print(f"[pose error]: {file} not exist")
            continue

        # log the number
        pose_count += len(data['people'])
        
        # 漏检测 只有1人的情况
        if len(data['people']) != 2:
            # drop this frame
            continue
        
        row0 = data['people'][0]['pose_keypoints_2d'] # x0,y0,c0,x1,y1,c1,... x24,y24,c24
        row1 = data['people'][1]['pose_keypoints_2d']

        # face_id对应 x较小 标记为0，x较大 标记为1 
        if(row0[0]>row1[0]):
            row0,row1 = row1,row0 # swap

        # 插入face_id
        # TODO del face_id no need anymore
        row0.insert(0,0)
        row1.insert(0,1)

        # 插入frame
        frame_id = int(file.split('_')[-2]) # 文件 frame_id

        # 去掉多余的最后一 frame 文件 否者会导致 frame 重复
        if frame_id % 5 != 0:
            continue

        frame_id = frame_id // 5 + 1 # 恢复frame id 抽帧后的 id
        row0.insert(0,frame_id)
        row1.insert(0,frame_id)

        pose_df0.loc[len(pose_df0)] = row0
        pose_df1.loc[len(pose_df1)] = row1

     # 将df的索引设置为frame_id,并且插值
    try:
        # set frame to int type
        pose_df0['frame'] = pose_df0['frame'].astype(int)
        pose_df1['frame'] = pose_df1['frame'].astype(int)
        min_fm, max_fm = pose_df0['frame'].min(), pose_df0['frame'].max()

        if(max_fm - min_fm >2):
            # 记录contribute 数据数量
            rpose_count += 2*pose_df0.shape[0]
            # 插值
            pose_df0 = pose_df0.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
            pose_df1 = pose_df1.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
        else:
            print(f"[pose error] in {feature_file}: not enough frames")
            # print(f"len:{len(face_df0)} and {len(face_df1)}")
            continue
    except:
        print(f"[pose error] in {feature_file}:unknown error in interpolation and reindex")
        continue

    ###############  pose low feature #######################
    
    # TODO select low feat
    # pose_low_feat = ['x','y']
    pose_low_feat = col_name[2:]
    pose_low_feat = [col for col in pose_low_feat if not col.startswith('c')] # filter all confidence

    # 每个人的low feat 的 vel and acc
    v_mean_0, v_var_0, a_mean_0, a_var_0 = get_vel_and_acc(pose_df0[pose_low_feat])
    v_mean_1, v_var_1, a_mean_1, a_var_1 = get_vel_and_acc(pose_df1[pose_low_feat])
    mean_0 = np.array(pose_df0[pose_low_feat].mean())
    mean_1 = np.array(pose_df1[pose_low_feat].mean())
    var_0 = np.array(pose_df0[pose_low_feat].var())
    var_1 = np.array(pose_df1[pose_low_feat].var())
    

    ###############  pose high feature #######################
    # TODO pose high feature

    # # @1 left and right shoulder distance vel → (mean + var) = 2
    # shoulder_lf_diff_0 = (pose_df0[['x2','y2']] - pose_df0[['x5','y5']]).abs()
    # shoulder_lf_diff_1 = (pose_df1[['x2','y2']] - pose_df1[['x5','y5']]).abs()

    # v_mean_lrs, v_var_lrs, a_mean_lrs, a_var_lrs = get_vel_and_acc(np.concatenate(shoulder_lf_diff_0,shoulder_lf_diff_1))

    # # @2 质心靠近程度
    # centroid_diff_x = np.abs(mean_0[0:-1:2].mean() - mean_1[0:-1:2].mean()) # 所有 x 平均(单数 index)
    # centroid_diff_y = np.abs(mean_0[1:-1:2].mean() - mean_1[1:-1:2].mean()) # 所有 y 平均(双数 index)


    # features_pose = np.array(pose_json_data)
    low_features_pose_0 = np.concatenate((v_mean_0, v_var_0, a_mean_0, a_var_0,mean_0,var_0))
    low_features_pose_1 = np.concatenate((v_mean_1, v_var_1, a_mean_1, a_var_1,mean_1,var_1))

    # 将记录视频特征
    low_video_features.append(np.concatenate((low_features_face_0, low_features_pose_0, low_features_face_1, low_features_pose_1))) # simply attach them all together
    high_video_features.append(np.concatenate((high_features_face, low_features_face_0, low_features_pose_0, low_features_face_1, low_features_pose_1)))


    # video 级别label
    video_labels.append(np.argmax(label_value.astype("int"),axis=0)) # one-hot to class id
    
    # # frame级别label
    # video_labels.append(np.array(labels))

X_low = low_video_features
X_high = high_video_features

y = video_labels


# 总共遍历的face特征和pose特征数
print(f"Total face feat:{face_count}, pose feat:{pose_count}")
# 读入的数量
print(f"Read in videos X:{len(X_low)}, face_feat:{rface_count},pose_feat:{rpose_count}")
# # drop
print(f"Drop {face_count - rface_count} face features ({1 - rface_count/face_count:.2f}%), {pose_count - rpose_count} pose features({1 - rpose_count/pose_count:.2f}%)")


[face error] in siblings_face_1024/VCAM_0011_22.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0011_29.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0011_103.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0011_144.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0010_8.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0010_29.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0010_55.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0010_108.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0010_112.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0009_14.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0009_17.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0009_19.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0009_24.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0009_31.csv: not enough frames
[fa

/tmp/ipykernel_562253/2068437869.py:170: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cos_angles)


[face error] in siblings_face_1024/VCAM_0018_7.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0018_59.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0018_101.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0018_126.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0017_10.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0017_59.csv: not enough frames
[face error] in siblings_face_1024/VCAM_0016_47.csv: not enough frames
[pose error] in siblings_face_1024/VCAM_0020.csv: not enough frames
[pose error]: VCAM_0020_1_000000000000_keypoints.json not exist
[pose error] in siblings_face_1024/VCAM_0020_1.csv: not enough frames
[pose error]: VCAM_0021_000000000000_keypoints.json not exist
[pose error] in siblings_face_1024/VCAM_0021.csv: not enough frames
[pose error]: VCAM_0021_1_000000000000_keypoints.json not exist
[pose error] in siblings_face_1024/VCAM_0021_1.csv: not enough frames
[face error] in siblings_face_1024/V

In [8]:
# 总共遍历的face特征和pose特征数
print(f"Total face feat:{face_count}, pose feat:{pose_count}")
# 读入的数量
print(f"Read in videos X:{len(X_low)}, face_feat:{rface_count},pose_feat:{rpose_count}")
# # drop
print(f"Drop {face_count - rface_count} face features ({1 - rface_count/face_count:.2f}%), {pose_count - rpose_count} pose features({1 - rpose_count/pose_count:.2f}%)")

Total face feat:936507, pose feat:192635
Read in videos X:3032, face_feat:694744,pose_feat:191272
Drop 241763 face features (0.26%), 1363 pose features(0.01%)


### 数据选择和预处理

In [9]:
X = high_video_features

检查 nan

In [17]:
idx = np.where(np.isnan(X).any(axis=1)==True)[0]
idx

array([940])

In [18]:
if len(idx) != 0:
    print(f"drop {len(idx)} rows with nan value)}")
    for i in idx:
        X = np.delete(X, i, axis=0)
        y = np.delete(y, i, axis=0)

In [19]:
X_norm = np.apply_along_axis(lambda x: x / np.linalg.norm(x), axis=0, arr=X)


In [21]:
np.where(np.isnan(X_norm).any(axis=1)==True)

(array([], dtype=int64),)

In [22]:
print(f"Read in videos y:{len(y)}")

Read in videos y:3031


维度

In [23]:
print(f"X:{len(X[0])}")

X:746


In [34]:
y.min()

0